#Setup


In [ ]:
!nvidia-smi

Thu May  6 00:07:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q transformers
!pip install -q stanza
!pip install -q sentencepiece
!pip install -q pytorch-transformers

In [ ]:
import os, sys
import glob
import time
import nltk
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from transformers import AlbertPreTrainedModel,AlbertModel,AlbertTokenizer
from transformers import BertTokenizer,RobertaTokenizer,XLNetTokenizer, XLNetModel
from transformers import XLNetPreTrainedModel


from pathlib import Path


import matplotlib.pyplot as plt

import torch.optim as optim

import torch.utils
import torch.utils.data
from torch.utils.data import Dataset, DataLoader

from typing import List, Tuple, Union

import transformers
from transformers import BertModel, BertConfig, BertTokenizer, BertPreTrainedModel
from transformers import Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput

In [ ]:
is_cuda = True
is_cuda = is_cuda and torch.cuda.is_available() # will turn off cuda if the machine doesnt have a GPU

In [ ]:
!mkdir data/
!mkdir EPIE_Corpus
!git clone https://github.com/prateeksaxena2809/EPIE_Corpus.git data/EPIE_Corpus

mkdir: cannot create directory ‘data/’: File exists
mkdir: cannot create directory ‘EPIE_Corpus’: File exists
fatal: destination path 'data/EPIE_Corpus' already exists and is not an empty directory.


In [ ]:
tokenizer=AlbertTokenizer.from_pretrained('albert-base-v2')
Robertatokenizer = RobertaTokenizer.from_pretrained('roberta-base')
XLNETtokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# 1. Dataloader


In [ ]:
#option_num = {"A": 0, "B": 1, "C": 2, "D": 3}

In [ ]:
# make sure you place this notebook at the project root
data_dir = Path.cwd() / 'data'
static_idioms_path = data_dir / 'EPIE_Corpus/Static_Idioms_Corpus'

static_candidates_file = static_idioms_path / 'Static_Idioms_Candidates.txt'
static_tags_file = static_idioms_path / 'Static_Idioms_Tags.txt'
static_words_file = static_idioms_path / 'Static_Idioms_Words.txt'


In [ ]:
words_text = static_words_file.read_text(encoding='utf-8')
candidates_text = static_candidates_file.read_text(encoding='utf-8')
tags_text = static_tags_file.read_text(encoding='utf-8')

In [ ]:
Candidate = pd.read_csv(static_candidates_file,header=None)
Tags = pd.read_csv(static_tags_file,header=None)

In [ ]:
#find the index range we want to get rid of for each sentence
index_list = []
for i,row in Tags.iterrows():
  cur_tag = row[0]
  split_cur_tag = cur_tag.split(' ')
  begin_idx = split_cur_tag.index('B-IDIOM')
  end_idx = len(split_cur_tag) - 1 - split_cur_tag[::-1].index('I-IDIOM')
  cur_tag_index = []
  #the begin and end index of the sentence we need to get rid of
  cur_tag_index.append(begin_idx)
  cur_tag_index.append(end_idx)
  index_list.append(cur_tag_index)

  #only for testing purpose
  if i < 5: 
    print(split_cur_tag)
    print(begin_idx)
    print(end_idx)
    print(cur_tag_index)
print(index_list)


['O', 'O', 'O', 'O', 'O', 'B-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'O']
5
9
[5, 9]
['O', 'O', 'O', 'O', 'B-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'O', 'O', 'O', 'O']
4
8
[4, 8]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
10
14
[10, 14]
['O', 'O', 'B-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'O']
2
6
[2, 6]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'I-IDIOM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
14
18
[14, 18]
[[5, 9], [4, 8], [10, 14], [2, 6], [14, 18], [6, 10], [4, 8], [1, 5], [4, 8], [24, 28], [14, 18], [40, 44], [1, 5], [1, 5], [8, 9], [2, 3], [24, 25], [14, 15], [29, 30], [1, 2], [1, 2], [11, 14], [10, 13], [25, 28], [7, 10], [21, 24], [22, 25], [8, 9], [26,

In [ ]:
f = open(static_words_file,'r')
countline = 0
del_sentence = []
for line in f:
  cur_line = line.split(' ')
  cur_del_index = index_list[countline]
  update_line = cur_line[0:cur_del_index[0]] + cur_line[(cur_del_index[1]+1):]
  del_sentence.append(update_line)
  #only for test and visualize
  if countline < 5:
    print(cur_line)
    print(update_line)

  
  countline += 1
  

['Anyway', ',', 'thanks', 'MKM', 'and', 'keep', 'up', 'the', 'good', 'work', '!\n']
['Anyway', ',', 'thanks', 'MKM', 'and', '!\n']
['Well', 'done', 'Steffi', '—', 'keep', 'up', 'the', 'good', 'work', 'and', 'keep', 'winning', '.\n']
['Well', 'done', 'Steffi', '—', 'and', 'keep', 'winning', '.\n']
['Please', 'use', 'it', 'as', 'a', 'reminder', 'to', 'ensure', 'that', 'you', 'keep', 'up', 'the', 'good', 'work', 'and', 'the', 'resolutions', 'you', 'have', 'started', '.\n']
['Please', 'use', 'it', 'as', 'a', 'reminder', 'to', 'ensure', 'that', 'you', 'and', 'the', 'resolutions', 'you', 'have', 'started', '.\n']
['Please', 'do', 'keep', 'up', 'the', 'good', 'work', '.\n']
['Please', 'do', '.\n']
['We', 'hope', 'you', 'find', 'the', 'current', 'issue', 'still', 'of', 'interest', 'and', 'we', 'plan', 'to', 'keep', 'up', 'the', 'good', 'work', 'during', 'the', 'coming', 'year', 'seeking', 'out', 'happenings/events/', 'stories', 'to', 'retell', 'for', 'you', ',', 'but', 'as', 'it', 'is', 'your'

In [ ]:
Candidate.head(1)

,0
0,keep up the good work


In [ ]:
Tags.head(1)[0].tolist()

['O O O O O B-IDIOM I-IDIOM I-IDIOM I-IDIOM I-IDIOM O']

In [ ]:
len(np.unique(Candidate))

359

In [ ]:
Candidate

,0
0,keep up the good work
1,keep up the good work
2,keep up the good work
3,keep up the good work
4,keep up the good work
...,...
21886,quality time
21887,quality time
21888,quality time
21889,quality time


In [ ]:
Tags

,0
0,O O O O O B-IDIOM I-IDIOM I-IDIOM I-IDIOM I-ID...
1,O O O O B-IDIOM I-IDIOM I-IDIOM I-IDIOM I-IDIO...
2,O O O O O O O O O O B-IDIOM I-IDIOM I-IDIOM I-...
3,O O B-IDIOM I-IDIOM I-IDIOM I-IDIOM I-IDIOM O
4,O O O O O O O O O O O O O O B-IDIOM I-IDIOM I-...
...,...
21886,O O O O B-IDIOM I-IDIOM O O O O
21887,O O O O O O O O O O O O O O O O O O O O O O O ...
21888,O O O B-IDIOM I-IDIOM O O O O O O O O O O O O O O
21889,O O O O O O O B-IDIOM I-IDIOM O O O O O O O O ...


In [ ]:
Tags.shape[0]

21891

#2 Label the data

## 2.1 Label the sentence without the candiadate and with candidate

In [ ]:
def addLabel(sentence, tokenizer):
  tokens = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)
  Strtokens = tokenizer.tokenize(sentence)
  Strtokens = ['[CLS]'] + Strtokens + ['[SEP]']
  # To Do: Padding to same length
  return  tokens, Strtokens


In [ ]:
def addLabel_unknown(sentence_list, sentence, tokenizer, index):
  tokens = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)
  sentence1_list = sentence_list[0:index]
  sentence2_list = sentence_list[index:]
  sentence1 = " ".join(sentence1_list)
  sentence2 = " ".join(sentence2_list)
  Strtoken1 = tokenizer.tokenize(sentence1)
  Strtoken2 = tokenizer.tokenize(sentence2)
  #Strtokens_unk = ['[CLS]'] + Strtokens[0:index] + ['[UNK]'] + Strtokens[index:] + ['[SEP]']
  Strtokens_unk = ['[CLS]'] + Strtoken1 + ['[UNK]'] + Strtoken2 + ['[SEP]']
  return tokens, Strtokens_unk

In [ ]:
se = 'Anyway , thanks MKM and keep up the good work !'
se_list = se.split(' ')
addLabel_unknown(se_list, se, XLNETtokenizer, 5)

(tensor([[11815,    17,    19,  3466,   414,   536,   692,    21,   435,    76,
             18,   195,   154,    17,   136,     4,     3]]),
 ['[CLS]',
  '▁Anyway',
  '▁',
  ',',
  '▁thanks',
  '▁M',
  'K',
  'M',
  '▁and',
  '[UNK]',
  '▁keep',
  '▁up',
  '▁the',
  '▁good',
  '▁work',
  '▁',
  '!',
  '[SEP]'])

In [ ]:
addLabel('Anyway , thanks MKM and keep up the good work !', tokenizer)

(tensor([[    2,  2774,    13,    15,  3669, 10804,    79,    17,   643,    71,
             14,   254,   170,    13,   187,     3]]),
 ['[CLS]',
  '▁anyway',
  '▁',
  ',',
  '▁thanks',
  '▁mk',
  'm',
  '▁and',
  '▁keep',
  '▁up',
  '▁the',
  '▁good',
  '▁work',
  '▁',
  '!',
  '[SEP]'])

In [ ]:
#add CLS and SEP, UNK to del_sentence
token_encode_del_list = []
del_sentence_list = []
del_sentence_id_list = []
for i in range(len(del_sentence)):
  cur_del_sentence = " ".join(del_sentence[i])
  cur_del_idx = index_list[i][0]
  #print(cur_del_idx)
  token_encode_del, cur_del_label = addLabel_unknown(del_sentence[i],cur_del_sentence, tokenizer,cur_del_idx)
  #only for test and visualize
  if i < 5:
    print(del_sentence[i])
    print(cur_del_sentence)
    print(cur_del_label)
    print(tokenizer.convert_tokens_to_ids(cur_del_label))
  token_encode_del_list.append(token_encode_del)
  del_sentence_list.append(cur_del_label)
  del_sentence_id_list.append(tokenizer.convert_tokens_to_ids(cur_del_label))
 

#print(token_encode_del_list)
#print(del_sentence_list)

['Anyway', ',', 'thanks', 'MKM', 'and', '!\n']
Anyway , thanks MKM and !

['[CLS]', '▁anyway', '▁', ',', '▁thanks', '▁mk', 'm', '▁and', '[UNK]', '▁', '!', '[SEP]']
[2, 2774, 13, 15, 3669, 10804, 79, 17, 1, 13, 187, 3]
['Well', 'done', 'Steffi', '—', 'and', 'keep', 'winning', '.\n']
Well done Steffi — and keep winning .

['[CLS]', '▁well', '▁done', '▁steff', 'i', '▁', '—', '[UNK]', '▁and', '▁keep', '▁winning', '▁', '.', '[SEP]']
[2, 134, 677, 25973, 49, 13, 1, 1, 17, 643, 1414, 13, 9, 3]
['Please', 'use', 'it', 'as', 'a', 'reminder', 'to', 'ensure', 'that', 'you', 'and', 'the', 'resolutions', 'you', 'have', 'started', '.\n']
Please use it as a reminder to ensure that you and the resolutions you have started .

['[CLS]', '▁please', '▁use', '▁it', '▁as', '▁a', '▁reminder', '▁to', '▁ensure', '▁that', '▁you', '[UNK]', '▁and', '▁the', '▁resolution', 's', '▁you', '▁have', '▁started', '▁', '.', '[SEP]']
[2, 2247, 275, 32, 28, 21, 14973, 20, 4062, 30, 42, 1, 17, 14, 4302, 18, 42, 57, 373, 13, 9

Token indices sequence length is longer than the specified maximum sequence length for this model (781 > 512). Running this sequence through the model will result in indexing errors


##2.2 Label the candidate data

In [ ]:
#add CLS and SEP to candidate
token_encode_list = []
candidate_label_list = []
for i,row in Candidate.iterrows():
  cur_candidate = row[0]
  # if i == 0:
  #   print(cur_candidate)
  token_encode, cur_candidate_label = addLabel(cur_candidate, tokenizer)
  token_encode_list.append(token_encode)
  candidate_label_list.append(cur_candidate_label)
print(token_encode_list)
print(candidate_label_list)


[tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[  2, 643,  71,  14, 254, 170,   3]]), tensor([[   2, 4606, 1671,    3]]), tensor([[   2, 4606, 1671,    3]]), tensor([[   2, 4606, 1671,    3]]), tensor([[   2, 4606, 1671,    3]]), tensor([[   2, 4606, 1671,    3]]), tensor([[   2, 4606, 1671,    3]]), tensor([[   2, 4606, 1671,    3]]), tensor([[    2,  3028,  1968,  2127,    13, 19361,     3]]), tensor([[    2,  3028,  1968

#*3. Sentence Parser

In [ ]:
import stanza
stanza.download('en') 

2021-05-06 00:08:39 INFO: Downloading default packages for language: en (English)...
2021-05-06 00:10:01 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
pipeline = stanza.Pipeline()

2021-05-06 00:10:01 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-05-06 00:10:01 INFO: Use device: gpu
2021-05-06 00:10:01 INFO: Loading: tokenize
2021-05-06 00:10:07 INFO: Loading: pos
2021-05-06 00:10:07 INFO: Loading: lemma
2021-05-06 00:10:07 INFO: Loading: depparse
2021-05-06 00:10:08 INFO: Loading: sentiment
2021-05-06 00:10:08 INFO: Loading: ner
2021-05-06 00:10:09 INFO: Done loading processors!


In [ ]:
doc = pipeline('Anyway , thanks MKM and keep up the good work !')

In [ ]:
print(doc.entities)

[{
  "text": "MKM",
  "type": "PERSON",
  "start_char": 16,
  "end_char": 19
}]


#4 Decoder and Cloze

## 4.1 Sentecnce encoder

In [ ]:
class XLNETForCloze(XLNetPreTrainedModel):
  def __init__(self, config, len_idiom_vocab, model_name='XLNetCloze'):
        super().__init__(config)
        self.model_name = model_name
        self.xlnet = XLNetModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.Linear = nn.Linear(config.hidden_size, 1)
        self.idiom_embedding = nn.Embedding(len_idiom_vocab, config.hidden_size)
        self.init_weights()

  def vocab(self,blank_states):
    return torch.einsum('bd,nd->bn', [blank_states, self.idiom_embedding.weight])

  def forward(self, input_ids, token_type_ids, attention_mask, positions, option_ids,
                inputs_embeds=None, options_embeds=None, compute_loss=False, targets=None):
    encoded_outputs = self.xlnet(input_ids,
                                  token_type_ids=token_type_ids,
                                  attention_mask=attention_mask,
                                  inputs_embeds=inputs_embeds)
    encoded_hidden, encoded_mems, encoded_attentions = encoded_output.last_hidden_state, encoded_output.mems, encoded_output.attentions
    blank_states = encoded_hidden[[i for i in range(len(positions))], positions]
    if option_ids is None and options_embeds is None:
      raise ValueError('Rerun the model')
    elif option_embeds is not None:
      encoded_options = options_embeds
    else:
      encoded_options = self.idiom_embedding(option_ids)
    result = torch.einsum('abc,ac->abc', emcoded_options, blank_states)
    over_logits = self.vocab(blank_states)
    output = self.dropout(result)
    logits = self.Linear(output).view(len(positions), -1)

    if compute_loss:
      crossEntropy = nn.CrossEntropyLoss()
      loss = crossEntropy(logits, targets)
      target = torch.gather(option_ids, dim=1, index= targets.unsqueeze(1))
      over_loss = crossEntropy(over_logits, target.squeeze(1))
      return loss, over_loss
    else:
      cond_logits = torch.gather(over_logits,dim=1, index= option_ids)
      return logits, over_logits, cond_logits

## 4.2 choice Dncoder

In [ ]:
from transformers import BertPreTrainedModel
class BertForClozeChoices(BertPreTrainedModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.model_name = model_name
        self.bert = BertPreTrainedModel.from_pretrained(model_name)

        # add head on top as necessary
        self.head_on_top = nn.Sequential()

    def forward(self, input_ids=None, attention_mask=None,
                token_type_ids=None, position_ids=None, head_mask=None,
                inputs_embeds=None, encoder_hidden_states=None,
                encoder_attention_mask=None, past_key_values=None,
                use_cache=None, output_attentions=None,
                output_hidden_states=None, return_dict=None):
        # All of the above keyworded arguments is available from the output
        # of a BertTokenizer
        model_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )

        # parse outputs
        last_hidden_state = model_output.get('last_hidden_state')
        pooler_output = model_output.get('pooler_output')

        return last_hidden_state

# 5 Experiment

In [ ]:
# make sure you place this notebook at the project root
words_sentences = words_text.splitlines()
candidates_sentences = candidates_text.splitlines()
tags_sentences = tags_text.splitlines()
print(words_sentences[:3])
print(candidates_sentences[:3])
print(tags_sentences[:3])

['Anyway , thanks MKM and keep up the good work !', 'Well done Steffi — keep up the good work and keep winning .', 'Please use it as a reminder to ensure that you keep up the good work and the resolutions you have started .']
['keep up the good work', 'keep up the good work', 'keep up the good work']
['O O O O O B-IDIOM I-IDIOM I-IDIOM I-IDIOM I-IDIOM O', 'O O O O B-IDIOM I-IDIOM I-IDIOM I-IDIOM I-IDIOM O O O O', 'O O O O O O O O O O B-IDIOM I-IDIOM I-IDIOM I-IDIOM I-IDIOM O O O O O O O']


In [ ]:
model_name = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(model_name)

masked_sentences = []
for sentence, bio_tags in zip(words_sentences, tags_sentences):
    sentence: str
    bio_tags: str
    # split the sentence and BIO-tags by word and convert into numpy arrays
    s = np.array(sentence.split())
    t = np.array(bio_tags.split())
    indices = np.arange(len(s))
    # we will keep only this for each idiom
    bag_start_index = indices[t == 'B-IDIOM']
    # we will discard every I-IDIOMs
    bag_outside_index = indices[t != 'I-IDIOM']

    s[bag_start_index] = bert_tokenizer.mask_token
    s = s[bag_outside_index]

    masked_sentences.append(s)

for l, s in zip(masked_sentences[:3], words_sentences[:3]):
    print(f'Masked sentence: {" ".join(l)}\nOriginal sentence: {s}\n')


Masked sentence: Anyway , thanks MKM and [MASK] !
Original sentence: Anyway , thanks MKM and keep up the good work !

Masked sentence: Well done Steffi — [MASK] and keep winning .
Original sentence: Well done Steffi — keep up the good work and keep winning .

Masked sentence: Please use it as a reminder to ensure that you [MASK] and the resolutions you have started .
Original sentence: Please use it as a reminder to ensure that you keep up the good work and the resolutions you have started .



In [ ]:
all_candidates = set(candidates_sentences)
candidate_idiom_indices_dict = dict()
for index, idiom in enumerate(all_candidates):
    candidate_idiom_indices_dict[idiom] = index

def idiom_to_one_hot(idiom: str, num_total_idioms: int) -> torch.Tensor:
    if idiom not in candidate_idiom_indices_dict:
        return torch.zeros(size=(num_total_idioms,), dtype=torch.float)
    i = candidate_idiom_indices_dict[idiom]
    one_hot = torch.zeros(size=(num_total_idioms,), dtype=torch.float)
    one_hot[i] = 1.0
    return one_hot
sorted(candidate_idiom_indices_dict, key=lambda i: candidate_idiom_indices_dict[i])

['ring fencing',
 'in the bag',
 'below the salt',
 'out of sight',
 'bolt from the blue',
 'date rape',
 'god forbid',
 'well read',
 'last but not least',
 'a sea change',
 'cheek by jowl',
 'castles in the air',
 'under canvas',
 'out on the town',
 'absence makes the heart grow fonder',
 'bricks and mortar',
 'big bucks',
 'snowed under',
 'on the horns of a dilemma',
 'safety in numbers',
 'caught in the crossfire',
 'a new lease of life',
 'in the zone',
 'on the same wavelength',
 'mea culpa',
 'above suspicion',
 'through thick and thin',
 'flesh and blood',
 'technically speaking',
 'dressed up to the nines',
 'first come first served',
 'close to home',
 'give me five',
 'north and south',
 'in the dock',
 'many happy returns',
 'just what the doctor ordered',
 'up in the air',
 'dead serious',
 'in limbo',
 'off the record',
 'come hell or high water',
 'under the weather',
 'how come',
 'dog days',
 'in cahoots with',
 'dead wood',
 'silver lining',
 'never mind',
 'in the 

In [ ]:

class EPIEStaticCorpus(torch.utils.data.Dataset):
    def __init__(self, words_file_path: str, tags_file_path: str,
                 candidate_file_path: str, mask_token='[MASK]'):
        # read in the data
        words_text = Path(words_file_path).read_text(encoding='utf-8')
        tags_text = Path(tags_file_path).read_text(encoding='utf-8')
        candidate_text = Path(candidate_file_path).read_text\
            (encoding='utf-8')

        """
            self.indices is used to map user supplied indices to indices used
             by the dataset.
             Example:
                 User index:    0, 3, 4, 5
                 self.indices:  99, 21, 33, 45, 78, 61
                 Mapped:        99, 45, 78, 61
        """
        self.indices = np.array([])
        self.all_candidates = set()

        self.words_sentences = np.array(words_text.splitlines())
        self.candidates_sentences = np.array(candidate_text.splitlines())
        self.tags_sentences = np.array(tags_text.splitlines())

        self.masked_sentences = np.array([])
        self.candidates_to_labels_dict = dict()

        """
        Masking the dataset
            In the section below we will construct the dataset by replacing idioms
            indicated in the tags file with a mask token. Regardless of the length of the
             idiom, the entire idiom will be replaced with 1 mask token.

            Example:

                Original Sentence:  Hey, keep   up  the good    work    !
                Tags:               O    B      I   I   I       I       O
                Masked Sentence:    Hey, [MASK] !
        """

        masked_sentences = []
        for sentence, bio_tags in zip(words_sentences, tags_sentences):
            sentence: str
            bio_tags: str
            # split the sentence and BIO-tags by word and convert into numpy arrays
            s = np.array(sentence.split())
            t = np.array(bio_tags.split())
            indices = np.arange(len(s))
            # we will keep only this for each idiom
            bag_start_index = indices[t == 'B-IDIOM']
            # we will discard every I-IDIOMs
            bag_outside_index = indices[t != 'I-IDIOM']

            s[bag_start_index] = mask_token
            s = s[bag_outside_index]

            masked_sentences.append(s)
        self.masked_sentences = np.array(masked_sentences)
        # set up the indices table
        self.indices = np.arange(len(self.masked_sentences))
        """
        Preparing labels
            We need to convert the candidates (answers) to labels for use in this
            supervised learning task. This can be done by treating this task as a
            K-classification problem where K equals the number of candidate answers
            across the entire dataset. Each candidate answer is given an index and one hot
            encoded using that index.
        """
        self.all_candidates = set(candidates_sentences)
        for index, idiom in enumerate(self.all_candidates):
            self.candidates_to_labels_dict[idiom] = index

    def shuffle(self):
        # simply shuffle the indices mapping table
        np.random.shuffle(self.indices)

    def collate(self, data):
        """
        Collates the list of context sentence - candidate answer label pairs
        into:
            Tuple[list of context sentences, list of candidate labels]
        :param data: a list of context sentence-candidate answer lable pairs
        usually generated using __getitems__ of this class.
        :return: a tuple in the form of (list of context sentences, candidate
         labels)
        """
        context_sentences, labels = [], []
        for row in data:
            s, l = row
            context_sentences.append(" ".join(*s))
            labels.extend(l)
        return context_sentences, labels

    def __len__(self):
        return len(self.masked_sentences)

    def __getitem__(self, indices) -> Tuple[np.ndarray, Union[int, np
        .ndarray]]:
        """
        Returns a tuple of:
            1. the masked word sentence
            2. the label
        :param indices: indices used for indexing. This can be a torch
        tensor, a python list, or a numpy ndarray.
        :return: a tuple of (word sentence, label).
        """
        if torch.is_tensor(indices):
            indices: torch.Tensor
            indices = indices.tolist()
        if isinstance(indices, int):
            indices = [indices]
        # use the user supplied indices to index the dataset
        indices = self.indices[indices]

        sentences = self.masked_sentences[indices]
        candidates = self.candidates_sentences[indices]
        labels = [self.candidates_to_labels_dict[c] for c in candidates]

        if len(indices) > 1:
            return sentences, np.array(labels)
        else:
            return sentences, labels
dataset = EPIEStaticCorpus(
    words_file_path=static_words_file,
    tags_file_path=static_tags_file,
    candidate_file_path=static_candidates_file
)
dataset[[0, 1, 2, 3]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


(array([array(['Anyway', ',', 'thanks', 'MKM', 'and', '[MASK]', '!'], dtype='<U6'),
        array(['Well', 'done', 'Steffi', '—', '[MASK]', 'and', 'keep', 'winning',
        '.'], dtype='<U7'),
        array(['Please', 'use', 'it', 'as', 'a', 'reminder', 'to', 'ensure',
        'that', 'you', '[MASK]', 'and', 'the', 'resolutions', 'you',
        'have', 'started', '.'], dtype='<U11'),
        array(['Please', 'do', '[MASK]', '.'], dtype='<U6')], dtype=object),
 array([196, 196, 196, 196]))

In [ ]:
class EPIEStaticCorpusSimple(Dataset):
    def __init__(
            self,
            words_file_path: str,
            tags_file_path: str,
            candidates_file_path: str,
            mask_token: str = '[MASK]'
    ):
        self.mask_token = mask_token
        self.masked_sentences = np.array([])
        self.candidate_sentences = np.array([])
        # index table used to map user supplied index to dataset indices
        self.indices = np.array([])

        self.candidates = set()
        self.num_candidates = len(self.candidates)
        self.candidate_to_label_dict = dict()

        # loads the data
        words_sentences = Path(words_file_path).read_text(encoding='utf-8').splitlines()
        tags_sentences = Path(tags_file_path).read_text(encoding='utf-8').splitlines()
        candidates_sentences = Path(candidates_file_path).read_text(encoding='utf-8').splitlines()

        """
        Masking the dataset
            In the section below we will construct the dataset by replacing idioms
            indicated in the tags file with a mask token. Regardless of the length of the
             idiom, the entire idiom will be replaced with 1 mask token.

            Example:

                Original Sentence:  Hey, keep   up  the good    work    !
                Tags:               O    B      I   I   I       I       O
                Masked Sentence:    Hey, [MASK] !
        """
        masked_sentences = []
        for sentence, bio_tags, candidate in zip(words_sentences, tags_sentences, candidates_sentences):
            sentence: str
            bio_tags: str
            # split the sentence and BIO-tags by word, convert them into numpy arrays and mask them
            s = np.array(sentence.split(), dtype=object)
            t = np.array(bio_tags.split(), dtype=object)
            indices = np.arange(len(s))
            # keep only the beginning of the idiom
            bag_start_index = indices[t == 'B-IDIOM']
            # discard the in part of the idiom
            bag_outside_indices = indices[t != 'I-IDIOM']
            s[bag_start_index] = mask_token
            s = s[bag_outside_indices]

            masked_sentences.append(' '.join(s))  # restore them into untokenized sentences
            candidates_sentences.append(candidate)

        self.masked_sentences = np.array(masked_sentences, dtype=object)
        self.candidate_sentences = np.array(candidates_sentences, dtype=object)
        # set up the indices table
        self.indices = np.arange(len(self.masked_sentences))

        """
        Preparing labels
            We need to convert the candidates (answers) to labels for use in this
            supervised learning task. This can be done by treating this task as a
            K-classification problem where K equals the number of candidate answers
            across the entire dataset. Each candidate answer is given an index and one hot
            encoded using that index.
        """
        self.candidates = set(candidates_sentences)
        for index, idiom in enumerate(self.candidates):
            self.candidate_to_label_dict[idiom] = index

    def __len__(self):
        return len(self.masked_sentences)

    def __getitem__(self, index):
        """
        Returns a tuple of:
            1. the masked word sentence
            2. the label
        :param indices: indices used for indexing. This can be a torch
        tensor, a python list, or a numpy ndarray.
        :return: a tuple of (word sentence, label).
        """
        context_sentence = self.masked_sentences[index]
        candidate = self.candidate_sentences[index]
        label = self.candidate_to_label_dict[candidate]
        return context_sentence, candidate, label

In [ ]:
if __name__ == '__main__':
    print("Setting up dataset...")
    dataset = EPIEStaticCorpusSimple(
        'data/EPIE_Corpus/Static_Idioms_Corpus/Static_Idioms_Words.txt',
        'data/EPIE_Corpus/Static_Idioms_Corpus/Static_Idioms_Tags.txt',
        'data/EPIE_Corpus/Static_Idioms_Corpus/Static_Idioms_Candidates.txt',
    )
    print("Setting up model...")
    model = BertForClozeChoices(BertConfig)
    model.train()

    optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
    criterion = nn.NLLLoss()

    loader = DataLoader(dataset, batch_size=8, num_workers=8, shuffle=True)

    print("Starting training...")
    num_epochs = 21
    for e in range(num_epochs):
        print(f'{"=" * 7} Epoch {e + 1} {"=" * 7}')
        epoch_loss = 0.0
        correct = 0.0
        count = 0
        print('Preparing data...')
        for index, sample in enumerate(loader):
            optimizer.zero_grad()

            context_sentences, candidates, labels = sample
            model_outs = model((context_sentences, candidates))
            correct += (all(torch.argmax(model_outs,dim=1) == torch.tensor(labels).to(model.torch_device)))
            count += 1
            loss = criterion(model_outs, torch.tensor(labels).to(model.torch_device))
            if index % 500 == 0:
                print(f"Batch {index + 1} loss: {loss.item()}")


            epoch_loss += loss.item()

            loss.backward()
            optimizer.step()
        
        print(f"Epoch {e + 1} loss: {epoch_loss} Accuracy: {correct/count}\n")

Setting up dataset...
Setting up model...


ValueError: ignored